In [2]:
from transformers import AutoTokenizer, AutoModel

model = AutoModel.from_pretrained("flaubert/flaubert_base_cased")
tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_base_cased")


Downloading: 100%|██████████| 1.46k/1.46k [00:00<00:00, 510kB/s]
Downloading: 100%|██████████| 528M/528M [00:09<00:00, 58.0MB/s] 
Some weights of the model checkpoint at flaubert/flaubert_base_cased were not used when initializing FlaubertModel: ['pred_layer.proj.bias', 'pred_layer.proj.weight']
- This IS expected if you are initializing FlaubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 24.8kB/s]
Downloading: 100%|██████████| 1.49M/1.49M [00:00<00:00, 2.12MB/s]
Downloading: 100%|██████████| 875k/875k [00:00<00:00, 1.66MB/s]


In [3]:
sentence = "Le chat mange une pomme."
token_ids = tokenizer.encode(sentence)
print(token_ids)

[0, 55, 4427, 8425, 30, 8242, 16, 1]


In [4]:
import torch

token_ids = torch.tensor([token_ids])
token_ids

tensor([[   0,   55, 4427, 8425,   30, 8242,   16,    1]])

In [5]:
outputs = model(token_ids)

In [6]:
outputs

BaseModelOutput(last_hidden_state=tensor([[[ 0.3711,  0.3744, -0.1044,  ...,  0.2788,  1.6172,  1.5743],
         [-0.3475, -0.4301, -1.2414,  ..., -0.7886, -0.2546,  0.2149],
         [-0.6473, -1.6146, -2.5066,  ...,  0.5556, -0.3404,  0.1025],
         ...,
         [-2.6466, -0.0552, -3.8672,  ..., -2.0639,  0.6117, -1.1268],
         [-1.1969, -1.1622,  1.4329,  ..., -2.3905, -0.9578,  1.3920],
         [-2.1017, -1.3640,  1.2869,  ..., -1.7466,  0.3996,  0.1256]]],
       grad_fn=<MulBackward0>), hidden_states=None, attentions=None)

In [9]:
last_layer = outputs[0]
print(last_layer.shape)

torch.Size([1, 8, 768])


In [10]:
cls_embedding = last_layer[:, 0, :]
print(cls_embedding.shape)
print(cls_embedding)

torch.Size([1, 768])
tensor([[ 3.7105e-01,  3.7435e-01, -1.0444e-01, -8.5996e-01, -5.1998e-01,
          3.7790e-01, -1.0253e+00, -2.1970e+00,  8.2422e-01, -1.5244e+00,
         -1.9517e+00,  2.8696e-01,  1.3622e+00,  8.3143e-01, -1.9403e-01,
         -1.0642e+00,  5.4059e-01,  8.2617e-01, -3.1513e-01,  9.1901e-01,
          1.3942e+00, -3.8312e-01,  4.8980e-01,  1.0286e+00, -1.4923e+00,
          4.6063e-01, -1.9550e+00, -2.5232e+00,  1.8005e-02, -1.6495e+00,
          4.0918e-01, -7.4830e-02, -6.3810e-01, -1.0454e+00,  2.2735e+00,
         -8.5973e-02, -1.5857e+00,  1.8843e-01,  1.0462e+00,  1.3925e-01,
         -4.7871e-01, -4.2056e-01, -1.6651e+00, -1.7205e+00, -9.3378e-02,
          7.6245e-01, -7.5222e-01,  7.4744e-01,  6.4842e-01, -6.1274e-01,
         -6.0831e-01, -8.9337e-03, -2.3029e-01, -3.7819e-01,  1.1821e+00,
         -2.3758e-01, -1.1971e+00,  2.0361e+00, -1.6421e+00, -5.5941e-01,
          1.1438e-01, -1.6972e+00,  2.1781e+00, -9.6148e-01,  3.2107e-01,
          6.8465e